In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Charger les données
df_sales = pd.read_csv('historical_sales.csv')  # Données de ventes
df_trends = pd.read_csv('social_media_trends.csv')  # Tendances des réseaux sociaux
df_products = pd.read_csv('new_products_catalog.csv')  # Nouveaux produits

# Agréger les données de vente
sales_agg = df_sales.groupby('product_id').agg({'sales': 'sum'}).reset_index()

#Nettoyage et transformation des tendances
tfidf = TfidfVectorizer(stop_words='english', max_features=100)
trend_vectors = tfidf.fit_transform(df_trends['trend_keywords'])

# Encodage des produits
product_vectors = tfidf.transform(df_products['product_description'])

# Création des features en faisant correspondre tendances et produits
from sklearn.metrics.pairwise import cosine_similarity

similarity_scores = cosine_similarity(product_vectors, trend_vectors)
trend_match_score = np.max(similarity_scores, axis=1)  # Meilleur match par produit
df_products['trend_match_score'] = trend_match_score

# Fusion avec les ventes historiques
df_train = df_products.merge(sales_agg, on='product_id', how='left').fillna(0)

# Entraînement du modèle
X = df_train[['trend_match_score']]
y = df_train['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = lgb.LGBMRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')

# Prédiction des ventes pour les nouveaux produits
df_products['predicted_sales'] = model.predict(df_products[['trend_match_score']])

# Classement des meilleurs produits
df_products = df_products.sort_values(by='predicted_sales', ascending=False)
print(df_products[['product_id', 'product_name', 'predicted_sales']].head(10))
